In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L1\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['MIDATL.csv', 'SOUTH.csv', 'WEST.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[2]))

,Net,Temperature
0,49769.768,-9.0
1,48714.914,-9.0
2,47880.328,-9.0
3,47380.142,-9.0
4,47241.559,-9.1
...,...,...
43819,49258.613,-1.3
43820,47886.874,-1.5
43821,46582.286,-1.7
43822,45132.982,-2.1


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[2]

zoneData = pd.read_csv(os.path.join(path, fileName))
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_WEST,Temp_WEST
0,49769.768,-9.0
1,48714.914,-9.0
2,47880.328,-9.0
3,47380.142,-9.0
4,47241.559,-9.1
...,...,...
43819,49258.613,-1.3
43820,47886.874,-1.5
43821,46582.286,-1.7
43822,45132.982,-2.1


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_WEST,Temp_WEST,Lag-2,Lag-7
0,62703.510,-18.9,54253.358,49769.768
1,61831.479,-19.3,52894.746,48714.914
2,61662.224,-19.7,52217.360,47880.328
3,61596.448,-20.2,52040.692,47380.142
4,62540.295,-20.6,52576.307,47241.559
...,...,...,...,...
43651,49258.613,-1.3,42981.431,42521.702
43652,47886.874,-1.5,42260.559,42381.205
43653,46582.286,-1.7,41074.417,42197.353
43654,45132.982,-2.1,39600.208,41556.164


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 13s 8ms/step - loss: 0.0151 - mape: 8811.5957 - mae: 0.0953 - val_loss: 0.0090 - val_mape: 55.7136 - val_mae: 0.0739
Epoch 2/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0104 - mape: 8053.8999 - mae: 0.0805 - val_loss: 0.0090 - val_mape: 59.0203 - val_mae: 0.0720
Epoch 3/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0088 - mape: 7785.9922 - mae: 0.0740 - val_loss: 0.0144 - val_mape: 51.9056 - val_mae: 0.0884
Epoch 4/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0079 - mape: 7450.9297 - mae: 0.0697 - val_loss: 0.0119 - val_mape: 48.1186 - val_mae: 0.0808
Epoch 5/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0067 - mape: 7505.4287 - mae: 0.0634 - val_loss: 0.0150 - val_mape: 44.0552 - val_mae: 0.0942
Epoch 6/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0060 - mape: 6708.6514 - mae: 0.0597 - val_loss: 0.0144

1105/1105 [==============================] - 8s 7ms/step - loss: 0.0029 - mape: 2663.7139 - mae: 0.0401 - val_loss: 0.0115 - val_mape: 40.0431 - val_mae: 0.0802
Epoch 49/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0028 - mape: 2918.9685 - mae: 0.0395 - val_loss: 0.0106 - val_mape: 45.1524 - val_mae: 0.0760
Epoch 50/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0028 - mape: 2931.5557 - mae: 0.0397 - val_loss: 0.0108 - val_mape: 44.2042 - val_mae: 0.0755
Epoch 51/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0029 - mape: 3237.8540 - mae: 0.0398 - val_loss: 0.0107 - val_mape: 40.5958 - val_mae: 0.0763
Epoch 52/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0027 - mape: 3222.9441 - mae: 0.0390 - val_loss: 0.0104 - val_mape: 42.3981 - val_mae: 0.0741
Epoch 53/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0027 - mape: 3400.0635 - mae: 0.0385 - val_loss: 0.0131 - val_m

1105/1105 [==============================] - 8s 7ms/step - loss: 0.0014 - mape: 1714.3585 - mae: 0.0281 - val_loss: 0.0112 - val_mape: 34.8071 - val_mae: 0.0786
Epoch 96/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0014 - mape: 1704.9021 - mae: 0.0283 - val_loss: 0.0101 - val_mape: 39.9337 - val_mae: 0.0754
Epoch 97/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0014 - mape: 1849.0828 - mae: 0.0278 - val_loss: 0.0096 - val_mape: 38.9591 - val_mae: 0.0727
Epoch 98/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0014 - mape: 1695.7852 - mae: 0.0275 - val_loss: 0.0098 - val_mape: 37.6183 - val_mae: 0.0736
Epoch 99/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0014 - mape: 1544.8610 - mae: 0.0276 - val_loss: 0.0102 - val_mape: 41.7748 - val_mae: 0.0752
Epoch 100/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0014 - mape: 1883.9821 - mae: 0.0274 - val_loss: 0.0088 - val_

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 848.1717329025269


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 6ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.3396658 , 0.27962273, 0.25314546, ..., 0.39647773, 0.37566495,
       0.34374663], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.48882585, 0.43512496, 0.39838085, ..., 0.4469962 , 0.41965907,
       0.3888862 ])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,48799.925,40892.062500
1,45952.921,37708.820312
2,44004.897,36305.101562
3,42769.465,35463.937500
4,42277.236,35267.750000
...,...,...
4363,49258.613,43472.480469
4364,47886.874,44075.800781
4365,46582.286,43904.000000
4366,45132.982,42800.589844


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

7.312563839099784